### 환경 데이터 편차평균 보간법 진행

In [207]:
import pandas as pd

# 데이터를 불러옵니다. 파일 경로를 적절히 설정해야 합니다.
df = pd.read_csv('2020/환경_최종.csv', encoding='ansi')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71528 entries, 0 to 71527
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   realTime      71528 non-null  object 
 1   name          71528 non-null  int64  
 2   do            71528 non-null  object 
 3   city          71528 non-null  object 
 4   item          71528 non-null  object 
 5   cul_time      71528 non-null  int64  
 6   Measure_time  71212 non-null  object 
 7   temp_ex       69060 non-null  float64
 8   s_am          71528 non-null  float64
 9   as_am         71528 non-null  float64
 10  temp_int      69060 non-null  float64
 11  hum_int       69060 non-null  float64
 12  CO2           69060 non-null  float64
 13  st            69060 non-null  float64
dtypes: float64(7), int64(2), object(5)
memory usage: 7.6+ MB


,realTime,name,do,city,item,cul_time,Measure_time,temp_ex,s_am,as_am,temp_int,hum_int,CO2,st
0,2020.9.15 0:00,11,충남,부여,딸기,1,2020.9.15 0:00,18.1,0.0,0.0,19.8,92.0,549.0,15.831841
1,2020.9.15 1:00,11,충남,부여,딸기,1,2020.9.15 1:00,18.2,0.0,0.0,19.6,93.0,520.0,15.831841
2,2020.9.15 2:00,11,충남,부여,딸기,1,2020.9.15 2:00,18.2,0.0,0.0,19.9,95.0,540.0,15.831841
3,2020.9.15 3:00,11,충남,부여,딸기,1,2020.9.15 3:00,18.4,0.0,0.0,20.0,96.0,531.0,15.831841
4,2020.9.15 4:00,11,충남,부여,딸기,1,2020.9.15 4:00,18.0,0.0,0.0,19.6,96.0,632.0,15.831841


In [208]:
df["name"].unique()

array([11, 16, 20, 23, 24, 25, 26, 31, 34, 38, 49, 52], dtype=int64)

In [209]:
df = df[df["name"] != 52]

In [210]:
# 데이터를 날짜 및 시간 열로 변환합니다.
df['realTime'] = pd.to_datetime(df['realTime'])

In [211]:
orig_df = df[(df['realTime'] >= '2020-10-05 23:00:00') * (df['realTime'] <= '2020-11-14 10:00:00')]

c:\Users\mool8\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


농가명, realTime, 외부온도, 내부온도, 상대습도, CO2, 토양온도 편차
- 어제 - 오늘

In [212]:
orig_df.columns

Index(['realTime', 'name', 'do', 'city', 'item', 'cul_time', 'Measure_time',
       'temp_ex', 's_am', 'as_am', 'temp_int', 'hum_int', 'CO2', 'st'],
      dtype='object')

In [213]:
# 농가, 시간으로 행을 가져와서 전날에서 빼기
trange = pd.date_range(start='2020-10-05 23:00:00', end='2020-11-14 10:00:00', freq="H").tolist()
cols = ['temp_ex', 'temp_int', 'hum_int', 'CO2', 'st']
result = pd.DataFrame(columns=["name", "realTime"]+cols)


for i in orig_df["name"].unique()[:-1] :
    sub_df = orig_df[orig_df["name"]==i]
    res = []
    for t in range(1,len(trange)) :
        print(t)
        temp = [i, trange[t]]
        v = (sub_df[sub_df["realTime"] == trange[t-1]][cols].values) - (sub_df[sub_df["realTime"] == trange[t]][cols].values)
        res.append(temp + v[0].tolist())

    res_temp = pd.DataFrame(res, columns=["name", "realTime"]+cols)
    result = pd.concat([result, res_temp])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
3

In [214]:
result

,name,realTime,temp_ex,temp_int,hum_int,CO2,st
0,11,2020-10-06 00:00:00,0.2,0.018245,-3.0,-18.0,0.000000
1,11,2020-10-06 01:00:00,0.2,0.018245,-1.0,6.0,0.000000
2,11,2020-10-06 02:00:00,0.5,0.018245,0.0,-18.0,0.000000
3,11,2020-10-06 03:00:00,1.3,0.018245,0.0,-33.0,0.000000
4,11,2020-10-06 04:00:00,0.5,0.018245,-2.0,26.0,0.000000
...,...,...,...,...,...,...,...
942,38,2020-11-14 06:00:00,0.5,0.300000,-0.1,-3.0,-0.026226
943,38,2020-11-14 07:00:00,-1.0,0.100000,-0.9,-4.0,-0.026226
944,38,2020-11-14 08:00:00,0.7,0.500000,-1.5,-7.0,-0.026226
945,38,2020-11-14 09:00:00,-2.4,-4.200000,8.8,-11.0,-0.026226


In [215]:
result = result[(result["name"] != 23) & (result["name"] != 25)]

In [220]:
result

,name,realTime,temp_ex,temp_int,hum_int,CO2,st
0,11,2020-10-06 00:00:00,0.2,0.018245,-3.0,-18.0,0.000000
1,11,2020-10-06 01:00:00,0.2,0.018245,-1.0,6.0,0.000000
2,11,2020-10-06 02:00:00,0.5,0.018245,0.0,-18.0,0.000000
3,11,2020-10-06 03:00:00,1.3,0.018245,0.0,-33.0,0.000000
4,11,2020-10-06 04:00:00,0.5,0.018245,-2.0,26.0,0.000000
...,...,...,...,...,...,...,...
942,38,2020-11-14 06:00:00,0.5,0.300000,-0.1,-3.0,-0.026226
943,38,2020-11-14 07:00:00,-1.0,0.100000,-0.9,-4.0,-0.026226
944,38,2020-11-14 08:00:00,0.7,0.500000,-1.5,-7.0,-0.026226
945,38,2020-11-14 09:00:00,-2.4,-4.200000,8.8,-11.0,-0.026226


In [217]:
result_mean = result.groupby(["realTime"])[cols].mean().reset_index()

In [218]:
result_mean

,realTime,temp_ex,temp_int,hum_int,CO2,st
0,2020-10-06 00:00:00,0.227537,-0.146911,-0.744398,-13.879412,-0.035625
1,2020-10-06 01:00:00,0.277537,0.103089,-0.094398,-13.379412,0.176875
2,2020-10-06 02:00:00,0.477537,0.415589,-0.006898,-8.504412,0.289375
3,2020-10-06 03:00:00,0.677537,0.640589,0.530602,-7.754412,0.326875
4,2020-10-06 04:00:00,0.877537,0.803089,0.418102,-6.129412,0.526875
...,...,...,...,...,...,...
942,2020-11-14 06:00:00,0.212500,-0.337500,-0.075000,-12.000000,-0.003278
943,2020-11-14 07:00:00,-0.025000,-0.175000,-0.712500,-20.000000,-0.140778
944,2020-11-14 08:00:00,-0.312500,-0.125000,-0.487500,-17.750000,-0.153278
945,2020-11-14 09:00:00,-1.225000,-1.512500,2.237500,28.750000,-0.640778


In [221]:
result_mean.to_csv("2020/환경데이터_편차평균.csv", index=False)

In [ ]:
result[result["realTime"] == "2020-10-06 00:00:00"]

,name,realTime,temp_ex,temp_int,hum_int,CO2,st
0,11,2020-10-06,0.200000,0.018245,-3.000000,-18.000000,0.000
0,16,2020-10-06,0.020294,0.006471,0.013235,-0.035294,0.015
0,20,2020-10-06,0.400000,-0.100000,-0.500000,-21.000000,-0.100
0,24,2020-10-06,0.200000,0.400000,1.031579,-17.000000,0.500
0,26,2020-10-06,0.600000,-0.200000,-1.000000,-24.000000,0.200
0,31,2020-10-06,0.000000,0.300000,-2.100000,-8.000000,0.500
0,34,2020-10-06,0.300000,-1.600000,0.000000,-20.000000,-1.400
0,38,2020-10-06,0.100000,0.000000,-0.400000,-3.000000,0.000


In [ ]:
result[result["realTime"] == "2020-10-06 00:00:00"]["temp_ex"].mean()

0.22753676499999975